In [60]:
import pandas as pd

In [61]:
df_train_feng = pd.read_csv('/usr/src/tp2-1/210215_tp2_train_feng.csv')

In [62]:
df_train_feng.head()

,Unnamed: 0,Region0,Region1,Region2,Region3,Region4,Territory0,Territory1,Territory2,Territory3,...,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Quote_Type,Delivery_Year,TRF,Stage
0,16104,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12369.0,0.0,2018.0,2.0,0.0
1,16105,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,12370.0,0.0,2016.0,20.0,0.0
2,16106,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,12371.0,0.0,2016.0,0.0,0.0
3,16107,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,12371.0,0.0,2016.0,0.0,0.0
4,16109,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,12372.0,0.0,2016.0,3.0,1.0


In [63]:
X = df_train_feng.drop(columns = 'Stage')
y = df_train_feng['Stage']

In [64]:
from sklearn.model_selection import TimeSeriesSplit, ParameterGrid
import numpy as np

In [65]:
from sklearn.ensemble import RandomForestRegressor

splits = TimeSeriesSplit(n_splits=3, max_train_size=365*2)
rfr = RandomForestRegressor()

# Create a dictionary of hyperparameters to search
rfr_grid = {"n_estimators": [10, 20, 30, 40, 50, 100], 
        'max_depth': [3, 5, 8, 10, 12], 
        'max_features': [4, 8, 16, 32, 59], 
        'random_state': [123]}

rfr_paramGrid = ParameterGrid(rfr_grid)

def TimeSplit_ModBuild(model, paramGrid, splits, X, y):
    from sklearn.model_selection import TimeSeriesSplit
    from sklearn.metrics import mean_squared_error

    #Loop over each time split and for each
    for train_index, val_index in splits.split(X):
        _X_train_ = X.iloc[train_index]
        _y_train_ = y.iloc[train_index]
        _X_val_ = X.iloc[val_index]
        _y_val_ = y.iloc[val_index]

        train_scores = []
        val_scores = []
        #models = []
        
        # Loop through the parameter grid, set the hyperparameters, and save the scores
        for g in paramGrid:
            model.set_params(**g)
            model.fit(_X_train_, _y_train_)
            p_train = model.predict(_X_train_)
            p_val = model.predict(_X_val_)
            score_train = np.mean(mean_squared_error(_y_train_, p_train))
            score_val = np.mean(mean_squared_error(_y_val_, p_val))
            train_scores.append(score_train)
            val_scores.append(score_val)
            #models.append(model)
            best_idx = np.argmin(val_scores)
            
        print("Best-Fold HyperParams:: ", paramGrid[best_idx])
        print("Best-Fold Train RMSE: ", train_scores[best_idx])
        print("Best-Fold Val RMSE: ",val_scores[best_idx])
        print("\n")
        
    #Return most recent model
    return train_scores, val_scores, best_idx


CV_rfr_tup = TimeSplit_ModBuild(rfr, rfr_paramGrid, splits, X, y)

Best-Fold HyperParams::  {'random_state': 123, 'n_estimators': 100, 'max_features': 59, 'max_depth': 10}
Best-Fold Train RMSE:  0.07892414742313567
Best-Fold Val RMSE:  0.17908104537957306


Best-Fold HyperParams::  {'random_state': 123, 'n_estimators': 20, 'max_features': 32, 'max_depth': 12}
Best-Fold Train RMSE:  0.1061013809476538
Best-Fold Val RMSE:  0.1916428958108206


Best-Fold HyperParams::  {'random_state': 123, 'n_estimators': 20, 'max_features': 59, 'max_depth': 12}
Best-Fold Train RMSE:  0.09438841658691703
Best-Fold Val RMSE:  0.21337172344198851




In [66]:
df_frio_frio_test = pd.read_csv('/usr/src/tp2-1/210215_tp2_test_feng.csv')

In [67]:
df_frio_frio_test

,Unnamed: 0,Region0,Region1,Region2,Region3,Region4,Territory0,Territory1,Territory2,Territory3,...,ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Quote_Type,Delivery_Year,TRF
0,718,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9996,1,1,0,0,0,11222,0,2019,0
1,719,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21310,1,1,1,1,0,11223,0,2019,1
2,726,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21437,1,1,1,1,0,11224,0,2019,1
3,725,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21434,1,1,1,1,0,11224,0,2019,1
4,720,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21313,1,1,1,1,0,11223,0,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,2545,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16339,1,1,0,0,0,12364,0,2019,0
2547,2546,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16345,1,1,0,0,0,12364,0,2019,0
2548,2547,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15218,1,1,0,0,0,12365,0,2019,0
2549,2548,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15224,1,1,1,1,0,12366,0,2019,0


In [68]:
best_rfr_idx = CV_rfr_tup[2]

In [69]:
best_rfr_grid = rfr_paramGrid[best_rfr_idx]

best_rfr = RandomForestRegressor()
best_rfr.set_params(**best_rfr_grid)


RandomForestRegressor(max_depth=12, max_features=59, n_estimators=20,
                      random_state=123)

In [70]:
best_rfr.fit(X, y)

RandomForestRegressor(max_depth=12, max_features=59, n_estimators=20,
                      random_state=123)

In [71]:
predicts = best_rfr.predict(df_frio_frio_test)

In [72]:
predicts

array([0.57636153, 0.51075633, 0.5011879 , ..., 0.49285612, 0.50929591,
       0.44896914])

In [76]:
output = pd.DataFrame({'Opportunity_ID':df_frio_frio_test.Opportunity_ID, 'Target': predicts})
output

,Opportunity_ID,Target
0,11222,0.576362
1,11223,0.510756
2,11224,0.501188
3,11224,0.501188
4,11223,0.510756
...,...,...
2546,12364,0.661200
2547,12364,0.636639
2548,12365,0.492856
2549,12366,0.509296


In [77]:
final=output.groupby('Opportunity_ID').mean()
final

,Target
Opportunity_ID,
10689,0.603025
10690,0.547861
10691,0.561362
10692,0.464678
10693,0.619216
...,...
12364,0.648920
12365,0.492856
12366,0.509296


In [78]:
final.to_csv('/usr/src/tp2-1/210216_random_forest_TimeSeriesSplit.csv')

In [ ]:
'''
    de aqui en adelante eran pruebas para ver si puedo determinar que features tiene mayor
    peso para mejorar el feng
'''

In [14]:
import matplotlib.pyplot as plt

In [15]:
rfr.feature_importances_

array([0.02848471, 0.        , 0.        , ..., 0.00563403, 0.02752726,
       0.03078427])

In [16]:
score

[[1, 110, 60, 5, 50, -0.07108437660405165],
 [2, 110, 60, 5, 50, -0.13560651190189854],
 [3, 110, 60, 5, 50, -0.04803250299787076],
 [4, 110, 60, 5, 50, -0.07579638483298745],
 [5, 110, 60, 5, 50, 0.0006023351269941468],
 [6, 110, 60, 5, 50, -0.04883778692138163],
 [7, 110, 60, 5, 50, 0.06611411272943679],
 [8, 110, 60, 5, 50, 0.0351684624852131],
 [9, 110, 60, 5, 50, 0.013380949354706573],
 [10, 110, 60, 5, 50, -0.062007784449307435]]

In [17]:
plt.bar(X.columns, rfr.feature_importances_)
plt.xlabel('features')
plt.ylabel('importance ')
plt.show()

KeyboardInterrupt: 